In [117]:
from operator import index

import pandas as pd
import numpy as np
import folium
import requests

from bs4 import BeautifulSoup

# pip install tqdm : progressBar 구현
from tqdm.notebook import tqdm
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from tqdm import tqdm

# jupyter nbconvert --to script coffeeStore.ipynb

In [118]:
# 크롬 드라이버 다운로드 :  https://googlechromelabs.github.io/chrome-for-testing/
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간 10초
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [119]:
# driver.maximize_window()

In [120]:
dataOut = './../dataOut/'

In [122]:
# 스타벅스) 매장 찾기 - 지역검색 클릭
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=loacale'
driver.get(starbucks_url) # 해당 페이지로 이동하기

In [124]:
starbucks_location = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_location))).click()

In [125]:
# 스타벅스 '서울' 링크 클릭
starbucks_seoul_selector = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_selector))).click()

In [126]:
# 스타벅스 '서울-전체' 링크 클릭
starbucks_seoul_all = '#mCSB_2_container > ul > li:nth-child(1) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all))).click()

In [127]:
# 스타벅스 html 코드를 파싱해서 html 파일에 기록합니다.
html = driver.page_source
filename = 'starbucks.html'
htmlfile = open(filename, mode='wt', encoding='UTF-8')
print(html, file = htmlfile)
htmlfile.close()
print(filename,'파일 생성됨')

starbucks.html 파일 생성됨


In [128]:
# 파싱된 결과를 BeautifulSoup 객체로 생성
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [129]:
# id 속성이 mCSB_3_container인 <div> 태그를 찾아주세요.
container = soup.find('div', attrs = {'id' : 'mCSB_3_container'})

# 그 아래에 있는 모든 <li> 태그를 찾아주세요
storeAll = container.find_all('li')
print(f'모든 매장 갯수 : {len(storeAll)}')

모든 매장 갯수 : 665


In [130]:
starbucksData = [] # 스타벅스 매장 리스트
for store in storeAll:
    # print(store)
    brand = '스타벅스'
    name = store.find('i').text
    address = store.find('p').text
    imsi = address.split(' ')
    sido = imsi[0]
    gungu = imsi[1]
    latitude = store['data-lat']
    longitude = store['data-long']

    starbucksData.append((brand, name, address, sido, gungu, latitude, longitude))
    # break

print(f'스타벅스 매장 갯수 : {len(starbucksData)}')
# print(starbucksData)

스타벅스 매장 갯수 : 665


In [131]:
sbColumns = ['브랜드','상호','주소','시도','군구','위도','경도']
sbDataFrame = pd.DataFrame(starbucksData)
sbDataFrame.columns = sbColumns
sbDataFrame.head()

,브랜드,상호,주소,시도,군구,위도,경도
0,스타벅스,리저브 매장 2번,서울특별시 강남구 언주로 425 (역삼동)1522-3232,서울특별시,강남구,37.501087,127.043069
1,스타벅스,리저브 매장 2번,서울특별시 강남구 강남대로 538 (논현동)1522-3232,서울특별시,강남구,37.510178,127.022223
2,스타벅스,리저브 매장 2번,서울특별시 강남구 강남대로 584 (논현동)1522-3232,서울특별시,강남구,37.5139309,127.0206057
3,스타벅스,리저브 매장 2번,서울특별시 강남구 테헤란로 125 (역삼동)1522-3232,서울특별시,강남구,37.499517,127.031495
4,스타벅스,리저브 매장 2번,서울특별시 강남구 남부순환로 2947 (대치동)1522-3232,서울특별시,강남구,37.494668,127.062583


In [132]:
sbFilename = dataOut + 'starbucks_file.csv'
sbDataFrame.to_csv(sbFilename, index=False, encoding='UTF-8')
print(f'{sbFilename} 파일이 저장되었습니다.')

./../dataOut/starbucks_file.csv 파일이 저장되었습니다.


In [133]:
# 위도, 경도 , 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {sbDataFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {sbDataFrame["경도"].isnull().sum()}')

위도 누락 데이터 갯수 : 0
경도 누락 데이터 갯수 : 0


In [134]:
print('# 서울시 구 갯수 파악')
seoul_gu_count = list(sbDataFrame['군구'].unique())
print(f'서울시 구 갯수 : {len(seoul_gu_count)}')
print(f'서울시 구 목록 : {seoul_gu_count}')

# 서울시 구 갯수 파악
서울시 구 갯수 : 25
서울시 구 목록 : ['강남구', '강북구', '강서구', '관악구', '광진구', '금천구', '노원구', '도봉구', '동작구', '마포구', '서대문구', '서초구', '성북구', '송파구', '양천구', '영등포구', '은평구', '종로구', '중구', '강동구', '구로구', '동대문구', '성동구', '용산구', '중랑구']


In [135]:
# 누락된 위도와 경도 정보는 kakao api를 이용하여 채워 넣도록 합니다.
# https://developers.kakao.com/
# kakao에 로그인하여 API 키 발급 받기
# 주의) 카카오 개발자 사이트에서 카카오맵-사용설정-On 상태로 변경해주세요

In [136]:
url_header = 'https://dapi.kakao.com/v2/local/search/address.json?query='
api_key = '46398254ab79950eddfd3bfb6e2d198c'
header = {'Authorization': 'KakaoAK ' + api_key}

In [137]:
# 주소를 입력 받아서 위도와 경도를 반환해주는 함수 구현하기

In [138]:
def getGeoCoder(address):
    result = ''
    url = url_header + address
    response = requests.get(url, headers= header)

    if response.status_code == 200:
        try:
            jsondata = response.json()["documents"][0]["address"]
            return jsondata['y'], jsondata['x']
        except Exception as err:
            print(err)
            return None
        # end try
    else:
        result = f'Error[{response.status_code}]'
    # end if
    return result

In [139]:
# 다음 주소에 대한 테스트를 진행합니다.
address01 = '서울 중랑구 망우로 460 (망우동)'
geoInfo01 = getGeoCoder(address01) # 잘 동작하는 주소
geoInfo01

address02 = '서울 노원구 한글비석로 409 (상계동) 1~2층'
geoInfo02 = getGeoCoder(address02) # NoneType이 리턴되는 주소
geoInfo02

list index out of range


In [140]:
'''
케이스 01
panLatTo('127.079680860838','37.5879908042663','6');fnMove();

케이스 02
panLatTo('0','0','11');fnMove();

케이스 03
panAddTo('서울 중랑구 신내로 211 (신내동)','8');fnMove();
'''

"\n케이스 01\npanLatTo('127.079680860838','37.5879908042663','6');fnMove();\n\n케이스 02\npanLatTo('0','0','11');fnMove();\n\n케이스 03\npanAddTo('서울 중랑구 신내로 211 (신내동)','8');fnMove();\n"

In [141]:
# 이디야 매장 정보
ediya_url = 'https://www.ediya.com/contents/find_store.html'
driver.get(ediya_url)

In [142]:
# 이디야 '매장명'과 '주소' 검색에서 '주소' 버튼 클릭
ediya_address_selector = '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_address_selector))).click()

In [143]:
# 서울시 구 목록을 순환하면서 크롤링
ediyaData = [] # 이디야 매장 정보 리스트
incorrect_address = [] # 위도/경도 추출에 문제가 있는 주소 리스트

ediya_search_inputbox = '#keyword' # 이디야 주소 검색 입력란
ediya_search_button = '#keyword_div > form > button' # 이디야 주소 검색(돋보기) 버튼

for gu in tqdm(seoul_gu_count):
    # 주소 검색 입력란 초기화
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_inputbox))).clear()
    # 주소 검색어 입력
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_inputbox))).send_keys(f'서울 {gu}')
    # 찾기(돋보기 버튼) 클릭
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_button))).click()
    # 이디야 매장 정보 수집
    html = driver.page_source

    if gu =='마포구': # 하나의 구 정보만 저장해보기
        gufile = open(f'서울 {gu}.html', 'wt', encoding='UTF-8')
        print(html, file=gufile)
        gufile.close()
        print(f'서울 {gu}.html 파일 생성 완료')
        # end if

     # 매장 정보는 id 속성이 placesList인 <ul> 태그 아래에 있습니다.
    soup = BeautifulSoup(html, 'html.parser')
    ul_tag = soup.find('ul', id='placesList')

    oneGuStoreList = ul_tag.find_all('li') # 구별 매장 리스트
    for store in oneGuStoreList:
        brand = '이디야'
        name = store.find('dt').text.strip()
        address = store.find('dd').text
        imsi = address.split(' ')
        sido = imsi[0]
        gungu = imsi[1]

        # 위도와 경도는 임시 비활성화
        geoInfoString = store.find('a')['onclick']
        print(geoInfoString)
        latitude = np.nan # 결측치
        longitude = np.nan

        # 위도와 경도 보정
        if geoInfoString.startswith('panLatTo'):
            if geoInfoString.startswith("panLatTo('0','0'"): # 잘못된 위도/경도
                # address를 사용하여 kakao api에서 물어보기
                geo_info = getGeoCoder(address)
                if geo_info != None:
                    latitude, longitude = geo_info
                else:
                    incorrect_address.append(address)
                    latitude = None
                    longitude = None
                # end if
            else: # 올바른 위도/경도
                latLong = geoInfoString.split("'")
                latitude = latLong[3]
                longitude = latLong[1]
                print(f'위도 : {latitude}, 경도 : {longitude}')
        else: # panAddTo로 시작하는 경우
            geo_info = getGeoCoder(address)
            if geo_info != None:
                latitude, longitude = geo_info
            else:
                incorrect_address.append(address)
                latitude = None
                longitude = None

        ediyaData.append((brand, name, address, sido, gungu, latitude, longitude))
    #     break # 삭제 예정
    # # pass
    # break # 삭제 예정
# end for

print(f'이디야 매장 갯수 : {len(ediyaData)}')

  4%|▍         | 1/25 [00:00<00:03,  7.11it/s]

panLatTo('0','0','0');fnMove();
panLatTo('127.0401601992311','37.51654171724045','1');fnMove();
위도 : 37.51654171724045, 경도 : 127.0401601992311
panLatTo('0','0','2');fnMove();
panLatTo('127.02810578707652','37.51408005446769','3');fnMove();
위도 : 37.51408005446769, 경도 : 127.02810578707652
panLatTo('127.05242928262568','37.50133876179308','4');fnMove();
위도 : 37.50133876179308, 경도 : 127.05242928262568
panAddTo('서울 강남구 도산대로37길 20 (신사동)','5');fnMove();
panLatTo('127.107863694133','37.4735487656314','6');fnMove();
위도 : 37.4735487656314, 경도 : 127.107863694133
panLatTo('0','0','7');fnMove();
panLatTo('127.06742226515867','37.488604436046536','8');fnMove();
위도 : 37.488604436046536, 경도 : 127.06742226515867
panLatTo('127.07764131137398','37.49418891219431','9');fnMove();
위도 : 37.49418891219431, 경도 : 127.07764131137398
panLatTo('127.061617564506','37.4940924880388','10');fnMove();
위도 : 37.4940924880388, 경도 : 127.061617564506
panAddTo('서울 강남구 논현동 ','11');fnMove();
panLatTo('127.06310437319587','37.5

  8%|▊         | 2/25 [00:02<00:33,  1.45s/it]

panLatTo('127.032981540064','37.51047297481296','26');fnMove();
위도 : 37.51047297481296, 경도 : 127.032981540064
panLatTo('127.08160067456867','37.49040201406986','27');fnMove();
위도 : 37.49040201406986, 경도 : 127.08160067456867
panLatTo('127.05035288418118','37.524855870334314','28');fnMove();
위도 : 37.524855870334314, 경도 : 127.05035288418118
panLatTo('127.049146860404','37.5200643058897','29');fnMove();
위도 : 37.5200643058897, 경도 : 127.049146860404
panLatTo('127.03312378880225','37.51499417378034','30');fnMove();
위도 : 37.51499417378034, 경도 : 127.03312378880225
panLatTo('127.033652198273','37.5131174134101','31');fnMove();
위도 : 37.5131174134101, 경도 : 127.033652198273
panLatTo('127.056118735013','37.4979490135961','32');fnMove();
위도 : 37.4979490135961, 경도 : 127.056118735013
panAddTo('서울 강북구 삼양로 501 (수유동)','0');fnMove();
panLatTo('127.016163948133','37.6406008108564','1');fnMove();
위도 : 37.6406008108564, 경도 : 127.016163948133
panLatTo('127.034460650104','37.6169375235603','2');fnMove();
위도 : 3

 12%|█▏        | 3/25 [00:03<00:29,  1.35s/it]

...

panAddTo('서울 동대문구 답십리로 264 (장안동)','17');fnMove();
panLatTo('127.05749583678211','37.58980550001833','18');fnMove();
위도 : 37.58980550001833, 경도 : 127.05749583678211
panLatTo('127.053432156481','37.5641084405355','0');fnMove();
위도 : 37.5641084405355, 경도 : 127.053432156481
panAddTo('서울 성동구 고산자로 339-2 (마장동)','1');fnMove();
panLatTo('0','0','2');fnMove();
panLatTo('0','0','3');fnMove();
panLatTo('127.04361144463','37.5501670203702','4');fnMove();
위도 : 37.5501670203702, 경도 : 127.04361144463
panLatTo('0','0','5');fnMove();
panAddTo('서울 성동구 성수이로 118 (성수동2가)','6');fnMove();
panLatTo('127.06190839848826','37.54487737059238','7');fnMove();
위도 : 37.54487737059238, 경도 : 127.06190839848826
panLatTo('0','0','8');fnMove();
panAddTo('서울 성동구 매봉길 12 (옥수동)','9');fnMove();
panAddTo('서울 성동구 왕십리광장로 17 (행당동)','10');fnMove();
panLatTo('0','0','11');fnMove();
panLatTo('0','0','12');fnMove();
panLatTo('0','0','13');fnMove();
panLatTo('0','0','14');fnMove();


 96%|█████████▌| 24/25 [00:49<00:02,  2.14s/it]

panLatTo('127.00692431290224','37.53494410336005','0');fnMove();
위도 : 37.53494410336005, 경도 : 127.00692431290224
panAddTo('서울 용산구 이촌로77길 19 (이촌동)','1');fnMove();
panAddTo('서울 용산구 후암로 93 (동자동)','2');fnMove();
panLatTo('0','0','3');fnMove();
panAddTo('서울 용산구 한강대로 289 (갈월동)','4');fnMove();
panLatTo('0','0','5');fnMove();
panLatTo('126.9611373908156','37.53584882357322','6');fnMove();
위도 : 37.53584882357322, 경도 : 126.9611373908156
panLatTo('126.956231240406','37.5337400268036','7');fnMove();
위도 : 37.5337400268036, 경도 : 126.956231240406
panLatTo('126.951642152118','37.5325054872953','8');fnMove();
위도 : 37.5325054872953, 경도 : 126.951642152118
panLatTo('126.964651687385','37.5254293060308','9');fnMove();
위도 : 37.5254293060308, 경도 : 126.964651687385
panLatTo('126.969413497587','37.5494639189725','10');fnMove();
위도 : 37.5494639189725, 경도 : 126.969413497587
panLatTo('126.96220762686157','37.540537976136086','11');fnMove();
위도 : 37.540537976136086, 경도 : 126.96220762686157
panAddTo('서울 용산구 원효로 257

100%|██████████| 25/25 [00:50<00:00,  2.01s/it]

이디야 매장 갯수 : 474


In [144]:
edColumns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도']
edDataFrame = pd.DataFrame(ediyaData)
edDataFrame.columns = edColumns
edDataFrame.head()

,브랜드,상호,주소,시도,군구,위도,경도
0,이디야,강남YMCA점,서울 강남구 논현동,서울,강남구,37.512645150688,127.030154778539
1,이디야,강남구청역아이티웨딩점,"서울 강남구 학동로 338 (논현동, 강남파라곤)",서울,강남구,37.51654171724045,127.0401601992311
2,이디야,강남나누리병원점,서울 강남구 언주로149길 5 (논현동),서울,강남구,37.5205092341033,127.033694770592
3,이디야,강남논현학동점,서울 강남구 논현로131길 28 (논현동),서울,강남구,37.51408005446769,127.02810578707652
4,이디야,강남대치점,"서울 강남구 역삼로 415 (대치동, 성진빌딩)",서울,강남구,37.50133876179308,127.05242928262568


In [147]:
print('주소 변환에 문제가 있던 항목들')
incorrect_address

['서울 동작구 사당로16가길 96 (사당동) 1,2층', '서울 영등포구 영등포로35길 19 (영등포동6가)']

In [149]:
# 콤마를 공백으로, 2칸 공백은 1칸으로 치환합니다.
# 이후 공백으로 분리후, 슬라이싱을 사용하여 0번째 부터 3번째 글자까지만 추출합니다.
correct_address = [addr.replace(',',' ').replace('  ',' ').split(' ')[0:4] for addr in incorrect_address]
correct_address

[['서울', '동작구', '사당로16가길', '96'], ['서울', '영등포구', '영등포로35길', '19']]

In [152]:
# split() 함수로 나누어진 주소를 join 함수로 다시 문자열을 합칩니다.
correct_address = [' '.join(addr) for addr in correct_address]
correct_address

['서울 동작구 사당로16가길 96', '서울 영등포구 영등포로35길 19']

In [154]:
# 보정이 된 주소 이름으로 Kakao API에게 위도/경도 추출을 요청합니다.
correction_geoinfo = [getGeoCoder(addr) for addr in correct_address]
correction_geoinfo

list index out of range


[('37.485107611205', '126.968823424325'), None]

In [145]:
edFilename = dataOut + 'ediya_file.csv'
edDataFrame.to_csv(edFilename, index=False, encoding='UTF-8')
print(f'{edFilename} 파일이 저장되었습니다.')

./../dataOut/ediya_file.csv 파일이 저장되었습니다.


In [146]:
# 위도, 경도 , 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {edDataFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {edDataFrame["경도"].isnull().sum()}')

위도 누락 데이터 갯수 : 2
경도 누락 데이터 갯수 : 2
